In [1]:
from sqlalchemy import create_engine
from dotenv import load_dotenv
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
import pandas as pd
import time

load_dotenv()

True

In [3]:
table_names = ["Trades Receivable","Trades Payable", "Working Capital"]
tables = {}
for tab in table_names:
    df= pd.read_excel("inputs/Databases/WCM_modified.xlsx", sheet_name=tab)
    for col in df.columns:
        if 'id' in col:
            df[col] = df[col].astype(str)
    new_tab = tab.lower().replace(" ","_")
    df.to_excel(f"inputs/Databases/versao_demo/{new_tab}.xlsx")
    tables[new_tab] = df

In [6]:
print(tables["trades_payable"].dtypes)

id                        object
id_trades                 object
month_year        datetime64[ns]
country                   object
trades_payable           float64
overdue                  float64
dpo                      float64
tr_ico_to_pay            float64
due_interval              object
dtype: object


In [4]:
tables["trades_receivable"].head()

,id,id_trades,month_year,country,trades_receivable,overdue,dso,sales,cei,art,due_interval
0,1,202405011,2024-05-01,Brasil,244592.88,72456.27,99.5,267281.99,0.89,110.27,1-90 Days
1,2,202405012,2024-05-01,Argentina,186457.45,6479.17,32.6,198693.25,0.96,34.12,Not Due
2,3,202405013,2024-05-01,Colombia,215411.34,8120.89,39.4,267274.03,0.56,42.31,Not Due
3,4,202405014,2024-05-01,Peru,163571.07,44496.47,76.6,168822.44,0.63,85.67,91-180 Days
4,5,202405015,2024-05-01,Chile,132761.53,11659.33,30.9,162979.40,0.54,29.60,1-90 Days


In [7]:
df = tables["trades_receivable"]
df['overdue_ratio'] = df['overdue'] / df['trades_receivable']
df.to_excel(f"inputs/Databases/versao_demo/trades_receivable.xlsx")

In [12]:
tables["trades_receivable"]["country"].unique()

array(['Brasil', 'Argentina', 'Colombia', 'Peru', 'Chile', 'Mexico',
       'Vanuatu', 'Coreia do Sul', 'Namíbia', 'Ilhas Salomão', 'Guam',
       'Ilhas Turcas e Caicos', 'Burquina Faso', 'Honduras', 'Granada',
       'Dhekelia', 'Mali', 'Portugal', 'Papua-Nova Guiné', 'Malávi',
       'Burundi', 'Congo-Kinshasa', 'Dinamarca', 'Senegal'], dtype=object)

In [8]:
tables["trades_payable"].head()

,id,id_trades,month_year,country,trades_payable,overdue,dpo,tr_ico_to_pay,due_interval
0,1,202405011,2024-05-01,Brasil,118690.22,38466.96,125.6,82228.29,1-90 Days
1,2,202405012,2024-05-01,Argentina,90279.29,2763.82,45.2,67390.82,Not Due
2,3,202405013,2024-05-01,Colombia,148144.95,32090.12,103.7,60477.78,Not Due
3,4,202405014,2024-05-01,Peru,93842.59,9010.86,67.4,57234.83,Not Due
4,5,202405015,2024-05-01,Chile,126152.85,23097.11,66.2,81335.94,91-180 Days


In [9]:
tables["working_capital"].head()

,id_trades,month_year,country,working_capital,due_interval
0,202405011,2024-05-01,Brasil,125902.66,1-90 Days
1,202405012,2024-05-01,Argentina,96178.16,Not Due
2,202405013,2024-05-01,Colombia,67266.39,Not Due
3,202405014,2024-05-01,Peru,69728.48,91-180 Days
4,202405015,2024-05-01,Chile,6608.68,1-90 Days


In [4]:
# Bancos de dados
db_name = "db/aa-finance-predict.db"
engine = create_engine(f"sqlite:///{db_name}")

In [5]:
# # Adicionando tabelas ao db
# for df_name in tables.keys():
#     df = tables[df_name]
#     for col in df.columns:
#         if "nnamed" in col:
#             df= df.drop(col, axis=1)
#     df.to_sql(df_name, engine, index=False)

In [6]:
db = SQLDatabase(engine)
db.get_usable_table_names()

['trades_payable', 'trades_receivable', 'working_capital']